In [1]:
pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.5 MB/s eta 0:00:00


In [2]:
pip install pynlp

In [3]:
pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from deep_translator import GoogleTranslator

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
df = pd.read_csv('dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2514 entries, 0 to 2513
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   conversation_id_str      2514 non-null   object 
 1   created_at               2513 non-null   object 
 2   favorite_count           2512 non-null   float64
 3   full_text                2513 non-null   object 
 4   id_str                   2512 non-null   object 
 5   image_url                780 non-null    object 
 6   in_reply_to_screen_name  655 non-null    object 
 7   lang                     2512 non-null   object 
 8   location                 1934 non-null   object 
 9   quote_count              2512 non-null   object 
 10  reply_count              2512 non-null   float64
 11  retweet_count            2512 non-null   object 
 12  tweet_url                2511 non-null   object 
 13  user_id_str              2511 non-null   float64
 14  username                

In [8]:
#Hapus Kolom yang tidak perlukan & Baris yang kosong pada Full Text
df = df[['username','full_text']]
df = df.dropna(subset=['full_text'])
df.sample(n=5)

,username,full_text
1284,fendyoesin,1. Skill Programming Skill teknik yang dimaksu...
2392,iamhasrizal,AI sangat berguna dan harus diraikan sebagai p...
913,UMonline,Merekayasa masa depan Malaysia dinamik bersama...
2157,dadarkeju,Paham deh jadi ini pada dasarnya emang cuma se...
1969,alisyarief,Militer Jepang Akan Berinvestasi pada AI Otoma...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2513 entries, 0 to 2513
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   username   2511 non-null   object
 1   full_text  2513 non-null   object
dtypes: object(2)
memory usage: 58.9+ KB


In [10]:
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Menghapus URL
    text = re.sub(r'@\w+', '', text)  # Menghapus mention
    text = re.sub(r'#\w+', '', text)  # Menghapus hashtag
    text = text.lower()
    words = word_tokenize(text)
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['tweet_clean'] = df['full_text'].apply(clean_text)
df = df.drop_duplicates()

In [11]:
df.head()

,username,full_text,tweet_clean
0,liviatoures,Maybe read this ss again. Penggunaan AI itu ha...,maybe read this ss again guna ai harmful bang...
1,ismailfahmi,Kata Dharma AI itu artificial intelligence. Da...,dharma ai artificial intelligence intelijen a...
2,AghaarXx,Melihat teman-teman di sekitar terlalu bergant...,teman gantung aplikasi benar kesal salah seba...
3,nonaked,ㅤ Yang paling kena dampaknya mungkin di sektor...,kena dampak sektor kayak
4,mosmarth,Bayangin urusan riil problema AI: - pengembang...,bayangin urus riil problema ai - kembang etis...


In [12]:
df['tweet_clean'] = df['tweet_clean'].str.replace(r'\d+', '', regex=True)  # Menghapus angka
df['tweet_clean'] = df['tweet_clean'].str.replace(r'[^\w\s]', '', regex=True)  # Menghapus tanda baca

<ipython-input-12-a88c8559c1b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_clean'] = df['tweet_clean'].str.replace(r'\d+', '', regex=True)  # Menghapus angka
<ipython-input-12-a88c8559c1b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_clean'] = df['tweet_clean'].str.replace(r'[^\w\s]', '', regex=True)  # Menghapus tanda baca


In [ ]:
def translate_text(text):
    translator = GoogleTranslator(source='id', target='en')
    return translator.translate(text)

df['english_text'] = df['tweet_clean'].apply(translate_text)

In [ ]:
# Menghitung Polarity Dan Labeling
df['sentiment'] = df['english_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)


def get_sentiment(polarity):
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

df['label'] = df['sentiment'].apply(get_sentiment)

In [ ]:
df.sample(5)

Cek Jumlah Data tiap Label

In [ ]:
jumlah_sampel_per_kelas = df['label'].value_counts()
jumlah_sampel_per_kelas


Cek Proporsi label / tiap kelas

In [ ]:
# prompt: check label proportion from df label

proporsi_label = df['label'].value_counts(normalize=True)
print(proporsi_label)


Plotting Jumlah Data Tiap Label

In [ ]:
import matplotlib.pyplot as plt


label_counts = df['label'].value_counts()

plt.figure(figsize=(8, 6))
plt.bar(label_counts.index, label_counts.values, color=['red', 'blue', 'green'])
plt.xlabel('Sentiment Label')
plt.ylabel('Number of Tweets')
plt.title('Distribution of Sentiment Labels')
plt.show()

In [ ]:
df.to_csv('final-preprocessing.csv')

Smote

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Pisahkan fitur dan label
X = df['tweet_clean']
y = df['label']

# Bagi data menjadi train dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Terapkan TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Terapkan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

# Cek distribusi label setelah SMOTE
print("Distribusi label setelah SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

# Sekarang X_train_resampled dan y_train_resampled siap digunakan untuk training model

TFIDF LANGSUNG

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Memisahkan data menjadi fitur dan label
X = df['tweet_clean']
y = df['label']

# Membagi dataset menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menggunakan TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words=None)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


NBC

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Melatih model Naive Bayes
model = MultinomialNB()
model.fit(X_train_resampled, y_train_resampled)

# Memprediksi pada set pengujian
y_pred = model.predict(X_test_vectorized)

# Evaluasi model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

param_grid = {'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0]}
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)
grid_search.fit(X_train_resampled, y_train_resampled)
print(grid_search.best_params_)


In [ ]:
# prompt: how to search best parameter for the tfidfvectorizer and multinomialNB?

from sklearn.model_selection import GridSearchCV

# Parameter untuk TF-IDF Vectorizer
tfidf_param_grid = {
    'max_features': [1000, 2000, 3000, 4000, 5000],
    'ngram_range': [(1, 1), (1, 2)],
    'use_idf': [True, False]
}

# Parameter untuk MultinomialNB
nb_param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0]
}

# Pipeline untuk menggabungkan TF-IDF dan MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# GridSearchCV untuk mencari parameter terbaik
param_grid = {
    'tfidf__' + key: value for key, value in tfidf_param_grid.items()
}
param_grid.update({
    'nb__' + key: value for key, value in nb_param_grid.items()
})

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Cetak parameter terbaik
print("Parameter terbaik:")
print(grid_search.best_params_)

# Evaluasi model dengan parameter terbaik
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
#Saving Model
import joblib
joblib.dump(model,open('nbmodel.pkl',"wb"))
joblib.dump(vectorizer, open('tfidf_vect.pkl',"wb"))